In [3]:
import os
os.environ["OPENAI_API_KEY"] = "Add your ChatGPT api key"

In [4]:
## One time code to create sql lite db using csv files 
'''
import pandas as pd
import sqlite3

each_match_records_df['date'] = pd.to_datetime(each_match_records_df['date'], format='%d-%m-%Y')

each_match_records_df = pd.read_csv('each_match_records.csv')
each_ball_records_df = pd.read_csv('each_ball_records.csv')

# Create a connection to your SQLite database
conn = sqlite3.connect("ipl_2023.db") 

# Push the first DataFrame into the database
each_match_records_df.to_sql("each_match_records", conn, if_exists="replace", index=False)

# Push the second DataFrame into the database
each_ball_records_df.to_sql("each_ball_records", conn, if_exists="replace", index=False)

# Close the database connection
conn.close()
'''

'\nimport pandas as pd\nimport sqlite3\n\neach_match_records_df[\'date\'] = pd.to_datetime(each_match_records_df[\'date\'], format=\'%d-%m-%Y\')\n\neach_match_records_df = pd.read_csv(\'each_match_records.csv\')\neach_ball_records_df = pd.read_csv(\'each_ball_records.csv\')\n\n# Create a connection to your SQLite database\nconn = sqlite3.connect("ipl_2023.db") \n\n# Push the first DataFrame into the database\neach_match_records_df.to_sql("each_match_records", conn, if_exists="replace", index=False)\n\n# Push the second DataFrame into the database\neach_ball_records_df.to_sql("each_ball_records", conn, if_exists="replace", index=False)\n\n# Close the database connection\nconn.close()\n'

In [5]:
from langchain_experimental.sql import SQLDatabaseChain
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", verbose=True)

DEFAULT_TABLES = [
    'each_match_records',
    'each_ball_records'
]

In [6]:
def get_prompt():
    _DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
    Use the following format:

    Question: "Question here"
    SQLQuery: "SQL Query to run"
    SQLResult: "Result of the SQLQuery"

    Answer: ""

    Only use the following tables:

    {table_info}

    Question: {input}"""

    PROMPT = PromptTemplate(
        input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
    )
    return PROMPT

In [7]:
def get_db():
    db = SQLDatabase.from_uri("sqlite:///ipl_2023.db",
                                  include_tables = DEFAULT_TABLES,
                                  sample_rows_in_table_info=2)
    return db

In [8]:
def answer_question(query):
    PROMPT = get_prompt()
    db = get_db()
    try:
        db_chain = SQLDatabaseChain.from_llm(llm, db, 
                                             prompt=PROMPT, 
                                             verbose=True, 
                                             return_intermediate_steps=True,
                                            #use_query_checker=True
                                             )
        result = db_chain(query)
        sql_cmd = None
        for step in result['intermediate_steps']:
            if 'sql_cmd' in step:
                sql_cmd = step['sql_cmd']
                break

        final_op = "{}\nSQL Command: < {} >".format(result['result'], sql_cmd)
        print(final_op)  
        return result['result']
    except Exception as e:
        return f"An error occurred: {str(e)}"   

In [9]:
answer_question("who won the final match")



> Entering new SQLDatabaseChain chain...
who won the final match
SQLQuery:SELECT winner
FROM each_match_records
WHERE match_type = 'Final'
SQLResult: [('Chennai Super Kings',)]
Answer:Chennai Super Kings
> Finished chain.
Chennai Super Kings
SQL Command: < SELECT winner
FROM each_match_records
WHERE match_type = 'Final' >


'Chennai Super Kings'

In [ ]:
answer_question("which player won most number of man of the match award")

In [11]:
answer_question("who scored maximum runs")



> Entering new SQLDatabaseChain chain...
who scored maximum runs
SQLQuery:SELECT batter, SUM(score) AS total_runs
FROM each_ball_records
GROUP BY batter
ORDER BY total_runs DESC
LIMIT 1
SQLResult: [('Shubman Gill', 916)]
Answer:Shubman Gill scored the maximum runs with a total of 916.
> Finished chain.
Shubman Gill scored the maximum runs with a total of 916.
SQL Command: < SELECT batter, SUM(score) AS total_runs
FROM each_ball_records
GROUP BY batter
ORDER BY total_runs DESC
LIMIT 1 >


'Shubman Gill scored the maximum runs with a total of 916.'

In [ ]:
answer_question("which player took most number of wickets")

In [ ]:
answer_question("Hello Bot") #error traping 